# Factoring Integers With Shor's Algorithm

This notebook is the distilled version of [Qiskit's own intro to Shor's Algorithm](https://qiskit.org/textbook/ch-algorithms/shor.html), for a super quick demo on getting started with quantum computing. 

It's not designed as an intorduction to quantum computing, or an introduction to Qiskit. The [Qiskit textbook](https://qiskit.org/textbook/preface.html) does a much better job of this. Instead, it is intended to show the ease with which quantum computing can be accessed and used today (1st April 2022).


## Using Qiskit

Let's first include our dependencies, as we would in any Python program.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
from math import gcd
np.random.seed(1) # This is to make sure we get reproduceable results

: 

## Defining our problem

An application of Shor's algorithm is to factorise number into their prime factors. For example, if we have the number 15

In [ ]:
N = 15

And we want to find its prime factors (which are 3 and 5) computationally.

Without quantum computers we would rely on expensive algorithms such as the [General Number Field Sieve](https://en.wikipedia.org/wiki/General_number_field_sieve), which is the current best algorithm for general prime factorisation. Instead, we'll encode our initial number N into a set of qbits and run it through a series of operations and get a possible answer. The circuit we'll use looks like the following

![Shor's Algorithm circuit for 8 qbits](quantum-circuit.png)


## Implementing the circuit in Python

First we'll need some building blocks for our code

In [ ]:
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [ ]:
def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     
    qc.x(3+n_count) 
    for q in range(n_count): 
        qc.append(c_amod15(a, 2**q), 
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count))
    qc.measure(range(n_count), range(n_count))
    # Simulate Results
    aer_sim = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj, memory=True).result()
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**n_count)
    print("Corresponding Phase: %f" % phase)
    return phase

In [ ]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

# Factorising an integer using a quantum circuit

Now, we can use our building blocks to attempt to find the prime factor of 15. The cell below repeats our search until at least one factor of 15 is found. You should try re-running the cell a few times to see how it behaves.

In [55]:
a = randint(2,N) # Pick a random number to start our search for factors
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    phase = qpe_amod15(a) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N) # Denominator should (hopefully!) tell us r
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 15)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0: # Check to see if guess is a factor
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = True


Attempt 1:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 2:
Register Reading: 10000000
Corresponding Phase: 0.500000
Result: r = 2
Guessed Factors: 3 and 5
*** Non-trivial factor found: 3 ***
*** Non-trivial factor found: 5 ***


You'll probably notice that the algorithm is non-deterministic, and you get diffeerent beahviour each time. That's because our results are probabilistic! 

# Take-aways

1. **The tools are very approachable is very approachable**
    It took me only an hour or so to package the demo. Little bit more to clean up an existing notebook into a minimal viable implementation. 
2. **Quantum computation can be integrated directly into current computation**
    Making existing composability tools (such as functions, modules, APIs, and so on) available for free for application to quantum computing
3. **Simulators can be used to develop skills**
    Quantum computing can be simulated with classical computers, which mean that we can develop the theoretical and practical skills now on this emerging technology.
    
# References

This entire example is lifted straight from the [Qiskit textbook](https://qiskit.org/textbook/ch-algorithms/shor.html). See also [Quantum Algorithm Implementation for Beginners](https://arxiv.org/pdf/1804.03719.pdf).
